# setup

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
plt.style.use('seaborn-talk')
import numpy as np
import os
import pandas as pd
import pickle
from IPython.display import clear_output
try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
bpl.output_notebook()

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
#                     filename="/tmp/caiman.log",
                    level=logging.INFO)

Loading BokehJS ...

# files & functions

In [2]:
caiman_data_path = 'C:/Users/ll357/Documents/CaImAn/demos/temp_data/'
tif_files = glob.glob(caiman_data_path + 'i1350_211222/' + "*70k.tif", recursive = True)
print(tif_files)

['C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\\i1350_211222_002_multipage_70k.tif', 'C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\\i1350_211222_003_multipage_70k.tif', 'C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\\i1350_211222_004_multipage_70k.tif']


In [3]:
def caiman_parameter_set(file_now):
    fr = 30                             # imaging rate in frames per second
    decay_time = 0.8                    # length of a typical transient in seconds
    # revised to account for gcamp6s, ~750 ms decay time in V1 L2/3: https://elifesciences.org/articles/51675
    # decay_time: Length of a typical transient in seconds. decay_time is an approximation of the time scale over which to expect a significant shift in the calcium signal during a transient. It defaults to 0.4, which is appropriate for fast indicators (GCaMP6f), slow indicators might use 1 or even more. However, decay_time does not have to precisely fit the data, approximations are enough.
    # V1 layer 2/3: rise and decay time constants for GCaMP6f are 50-100 ms and 200-300 ms; and for GCaMP6s 150-200 ms and ~750 ms (4-5 APs, Figure 3C). https://elifesciences.org/articles/51675

    # motion correction parameters
    strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
    overlaps = (24, 24)         # overlap between pathes (size of patch strides+overlaps)
    max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
    max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
    pw_rigid = True             # flag for performing non-rigid motion correction

    # parameters for source extraction and deconvolution
    p = 1                       # order of the autoregressive system
    gnb = 2                     # number of global background components
    merge_thr = 0.85            # merging threshold, max correlation allowed
    rf = 15                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
    stride_cnmf = 6             # amount of overlap between the patches in pixels
    K = 4                       # number of components per patch
    gSig = [4, 4]               # expected half size of neurons in pixels 
    # kept 4 according to snipaste pixel measurement
    # adjust cell size bc of zoom change around 2021-09

    method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
    ssub = 1                    # spatial subsampling during initialization
    tsub = 1                    # temporal subsampling during intialization

    # parameters for component evaluation
    min_SNR = 2.0               # signal to noise ratio for accepting a component
    rval_thr = 0.90             # space correlation threshold for accepting a component # adjusted
    rval_lowest = 0.0           # added to eliminate fake overlapping neurons, default -1
    cnn_thr = 0.99              # threshold for CNN based classifier
    cnn_lowest = 0.50           # neurons with cnn probability lower than this value are rejected # adjusted
    
    opts_dict = {'fnames': file_now,
                'fr': fr,
                'decay_time': decay_time,
                'strides': strides,
                'overlaps': overlaps,
                'max_shifts': max_shifts,
                'max_deviation_rigid': max_deviation_rigid,
                'pw_rigid': pw_rigid,
                'p': p,
                'nb': gnb,
                'rf': rf,
                'K': K, 
                'stride': stride_cnmf,
                'method_init': method_init,
                'rolling_sum': True,
                'only_init': True,
                'ssub': ssub,
                'tsub': tsub,
                'merge_thr': merge_thr, 
                'min_SNR': min_SNR,
                'rval_thr': rval_thr,
                'use_cnn': True,
                'min_cnn_thr': cnn_thr,
                'cnn_lowest': cnn_lowest}

    opts = params.CNMFParams(params_dict=opts_dict)
    
    return opts

In [4]:
# def cluster_start():
#     #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
#     if 'dview' in locals():
#         cm.stop_server(dview=dview)
#     c, dview, n_processes = cm.cluster.setup_cluster(
#         backend='local', n_processes=None, single_thread=False)

In [5]:
def corr_img_save(file_now):
    Cn = cm.local_correlations(images.transpose(1,2,0))
    f = open(file_now[:-4] + 'Cn.pckl', 'wb')
    pickle.dump(Cn, f)
    f.close()

# run

In [ ]:
for file_now in tif_files:
    opts = caiman_parameter_set(file_now)
    clear_output(wait=True)
    print(file_now[:-4])
    
    c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)
    print('start fitting caiman')
    cnm1 = cnmf.CNMF(n_processes, params=opts, dview=dview)
    cnm1.fit_file(motion_correct=True)
    
    corr_img_save(file_now)
    print(len(cnm1.estimates.idx_components))
    print(len(cnm1.estimates.idx_components_bad))
    cnm1.estimates.detrend_df_f(quantileMin=8, frames_window=250) # generates cnm.F_dff
    
    cnm1.save(file_now[:-4] + '.hdf5')
    cm.stop_server(dview=dview)

C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\i1350_211222_002_multipage_70k


      204215 [params.py:                 set():972] [41556] Changing key n_processes in group patch from 1 to 63


start fitting caiman


      205435 [motion_correction.py:motion_correct_pwrigid():355] [41556] Generating template by rigid motion correction
      283948 [motion_correction.py:motion_correction_piecewise():3149] [41556] ** Starting parallel motion correction **
